In [ ]:
# |default_exp socket

In [ ]:
# |export
import json
import logging
import socket
import struct
from collections import OrderedDict
from threading import Thread
from time import sleep, time

from fastcore.foundation import L, store_attr

In [ ]:
# |export
class SimpleValueParser:
    def __init__(self, name, format):
        store_attr()
        self.struct = struct.Struct(format)

    def __call__(self, conn):
        (value,) = self.struct.unpack(conn.recv(self.struct.size))
        return self.name, value


class ListValueParser:
    def __init__(self, name, format):
        store_attr()
        self.structs = [struct.Struct(f) for f in format]

    def __call__(self, conn):
        values = []
        for s in self.structs:
            (value,) = s.unpack(conn.recv(s.size))
            values.append(value)
        return self.name, values


def build_parser(name, format):
    if isinstance(format, list):
        return ListValueParser(name, format)
    return SimpleValueParser(name, format)


class TelemetryParser:
    def __init__(self, conn):
        store_attr()
        self.config = json.loads(
            conn.recv(2048).decode(), object_pairs_hook=OrderedDict
        )
        logging.debug("Telemetry parser config received.")
        self.parsers = L(build_parser(*item) for item in self.config.items())

    def __call__(self):
        telemetry = dict(self.parsers.map(lambda parser: parser(self.conn)))
        telemetry["_updatetime"] = time()
        return telemetry

In [ ]:
# |export
class Socket(Thread):
    def __init__(self):
        super().__init__()
        self.connected = False
        self.previous_telemetry = {}
        self.telemetry = {}
        self.should_stop = False
        self.port = 0
        self.start()
        while not self.port:
            sleep(0.001)

    def run(self):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.bind(("127.0.0.1", self.port))
            self.port = s.getsockname()[1]
            logging.debug(
                f"Listening for OpenPlanet connection on port {self.port} ..."
            )
            s.listen()
            self.connection, _ = s.accept()
            with self.connection as conn:
                self.connected = True
                logging.debug("Connection with OpenPlanet established.")
                try:
                    self.parser = TelemetryParser(conn)
                    while not self.should_stop:
                        previous_telemetry = self.telemetry
                        self.telemetry = self.parser()
                        # Setting previous_telemetry after avoids race condition where telemetry and previous_telemetry are the same
                        self.previous_telemetry = previous_telemetry
                except KeyboardInterrupt:
                    logging.info(
                        "Received KeyboardInterrupt, closing socket with OpenPlanet"
                    )
                except (socket.error, struct.error) as e:
                    logging.exception("OpenPlanet socket error")
                    pass
        self.connected = False
        logging.debug("Connection with OpenPlanet closed or lost.")

    def __del__(self):
        self.connected = False
        self.should_stop = True
        self.join()

    def send_command(self, *args):
        command = f"{'|'.join(args)};"
        if self.connected:
            self.connection.sendall(command.encode())
        else:
            logging.error(
                f"Can't send the command '{command}' : the socket is not connected to the game."
            )

In [ ]:
# |hide
# |eval: false
from nbdev.doclinks import nbdev_export

nbdev_export()